You need to install PyWavelet if necessary: pip install pywavelet

In [ ]:
import pywt
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 10)

Visualization functions:

In [ ]:
def imshowgray(im, vmin=None, vmax=None):
    plt.imshow(im, cmap=plt.get_cmap('gray'), vmin=vmin, vmax=vmax)

    
def wavMask(dims, scale):
    sx, sy = dims
    res = np.ones(dims)
    NM = np.round(np.log2(dims))
    for n in range(int(np.min(NM)-scale+2)//2):
        res[:int(np.round(2**(NM[0]-n))), :int(np.round(2**(NM[1]-n)))] = \
            res[:int(np.round(2**(NM[0]-n))), :int(np.round(2**(NM[1]-n)))]/2
    return res


def imshowWAV(Wim, scale=1):
    plt.imshow(np.abs(Wim)*wavMask(Wim.shape, scale), cmap = plt.get_cmap('gray'))

    
def coeffs2img(LL, coeffs):
    LH, HL, HH = coeffs
    return np.vstack((np.hstack((LL, LH)), np.hstack((HL, HH))))


def unstack_coeffs(Wim):
        L1, L2  = np.hsplit(Wim, 2) 
        LL, HL = np.vsplit(L1, 2)
        LH, HH = np.vsplit(L2, 2)
        return LL, [LH, HL, HH]

    
def img2coeffs(Wim, levels=4):
    LL, c = unstack_coeffs(Wim)
    coeffs = [c]
    for i in range(levels-1):
        LL, c = unstack_coeffs(LL)
        coeffs.insert(0,c)
    coeffs.insert(0, LL)
    return coeffs
    
    
def dwt2(im):
    coeffs = pywt.wavedec2(im, wavelet='db4', mode='per', level=4)
    Wim, rest = coeffs[0], coeffs[1:]
    for levels in rest:
        Wim = coeffs2img(Wim, levels)
    return Wim


def idwt2(Wim):
    coeffs = img2coeffs(Wim, levels=4)
    return pywt.waverec2(coeffs, wavelet='db4', mode='per')

Before we dive into wavelet transforms, we need a nice image to perform the tests. The provided brain.npz file from the webpage has a very pretty brain image (note it is complex-valued!). Run the cell below to load this and other data, explained later.

In [ ]:
data = np.load('brain.npz')
im, mask_unif, mask_vardens, pdf_unif, pdf_vardens = \
data['im'], data['mask_unif'], data['mask_vardens'], data['pdf_unif'], data['pdf_vardens'], 

Here's an example of how to compute a Daubechies wavelet transform, display it, and reconstruct it again:

In [ ]:


Wim = dwt2(im)
im2 = idwt2(Wim)

plt.subplot(1,3,1)
imshowgray(np.abs(im))
plt.title('Original')

plt.subplot(1,3,2)
imshowWAV(Wim)
plt.title('DWT')

plt.subplot(1,3,3)
imshowgray(np.abs(im2))
plt.title('Reconstruction')

print('Reconstruction error:', np.linalg.norm(im - im2))



### Wavelet thresholding

Threshold the wavelet coefficients retaining only the largest 20% of the coefficients. Plot the masked wavelet coefficients. What has been thresholded?

### Sampled Fourier
An important aspect of random frequency-domain sampling is matching the power spectrum of the image. Since the energy in many images is concentrated in lower spatial frequencies, more samples should be allocated there. We have provided two 3-fold undersampling masks for you. The random masks are in the variables mask_unif and mask_vardens and were drawn from probability density functions (PDF) given by the variables pdf_unif and pdf_vardens, respectively. Compute the 2D Fourier transform of the image using a centered 2D FFT. Display the mask and the pdfs.

Multiply by the uniform mask, divide by the appropriate PDF (called density compensation), and compute the zero-filled Fourier transform:

In [ ]:
def fft2c(x):
    return 1 / np.sqrt(np.prod(x.shape)) * np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(x)))

def ifft2c(y):
    return np.sqrt(np.prod(y.shape)) * np.fft.ifftshift(np.fft.ifft2(np.fft.fftshift(y)))

In [ ]:
M = fft2c(im);
Mu = (M * mask_vardens) / pdf_vardens;
imu = ifft2c(Mu);

Display the image and the difference image compared to original image. Is the aliasing white random noise? Repeat for the variable density mask. What happened now? Both use a similar number of samples, but which gives you a better reconstruction?

### 2D POCS

Extend your 1D POCS algorithm for 2D images. Add another step of computing the wavelet transform before the soft-thresholding and the inverse wavelet transform after the soft-thresholding. Make sure that your SoftThresh function works for complex-valued data, as was mentioned earlier.

Reconstruct the images from both the uniform and the variable density under-sampled data. First get an idea of reasonable values for λ

by examining what would be thresholded. You can do this using

Wimu = dwt2(imu)
imshowgray(abs(Wimu) > lambda)

Don't use imshowWAV for this, since it will scale the different wavelet levels differently. You want a significant number of coefficients to be below λ

, but not so many that too much detail will be lost!

Start with the variable density data, and experiment with several values of λ
. You should only need about 20 iterations, but start with fewer while you convince yourself it is working! Compare the result after soft-thresholding to a zero-filled density compensated reconstruction, the original image, and a the original image soft-thresholded. As an initial image to the POCS, use a zero-filled density compensated reconstruction, as it will converge faster. Show the image, and the difference image for the λ

you find the most effective.

Then try the uniform density data. Run for at least 50-100 iterations, since this converges slowly. If you want to speed up the convergence, start with a relatively large λ
so that the recon will converge rapidly. Then, decrease λ using the previous recon as an initial image. For example, you might divide λ by two every 10 or 20 iterations (this is called continuation). Show the image, and the difference image for the λ (or the final λ if you use a sequence) that you find the most effective. Don’t spend too much time on this, the point should be clear by now.